Requirements

In [ ]:
#!pip install -U ipykernel
!pip install -q streamlit
!pip install pyngrok

     |████████████████████████████████| 8.2MB 6.5MB/s 
     |████████████████████████████████| 112kB 41.8MB/s 
     |████████████████████████████████| 163kB 40.5MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 4.2MB 35.4MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 122kB 42.6MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.4 which is incompatible.
     |████████████████████████████████| 747kB 5.3MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19246 sha256=94b6db3a17b7530032557d0baf4eab93c45c32c7cc4d9d5d81987b8e47973af9
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok


Download of the Keras model

In [ ]:
!curl -L "https://api.jl-workshop.com/v2/csdlg/download?h=1ae6bd0bb4&url=https%3A%2F%2Fwww.mediafire.com%2Ffile%2F26ld1azugwf4sr0%2Fvgg16_model_final.zip%2Ffile" > dataset.zip; unzip dataset.zip; rm dataset.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   182  100   182    0     0   1978      0 --:--:-- --:--:-- --:--:--  1978
100   121  100   121    0     0    102      0  0:00:01  0:00:01 --:--:--   111
100 52.7M  100 52.7M    0     0  16.8M      0  0:00:03  0:00:03 --:--:-- 38.0M
Archive:  dataset.zip
  inflating: vgg16_model_final.h5    


Streamlit application saved as UniverseClassifier.py

In [ ]:
%%writefile UniverseClassifier.py

import tensorflow as tf
import numpy as np
import streamlit as st
from PIL import Image
import requests
from io import BytesIO

#st.set_option( 'deprecation. showfileuploaderEncoding', False)
st.title("Universe Image Classifier")
st.text("Provide URL of Image for image classification")
st.text("The system detects if the image contains one of these classes: Galaxy, Nebula, Planet")

@st.cache(allow_output_mutation=True) 
def load_model():
  model = tf.keras.models.load_model('/content/vgg16_model_final.h5')
  return model

with st.spinner('Loading Model Into Memory....'):
  model = load_model()

classes=['Galaxy', 'Nebula', 'Planet', '404 Not found']

def scale(image):
  image = tf.cast(image, tf.float32)
  image /= 255.0
  return tf.image.resize(image, [224,224])

def decode_img(image):
  img = tf.image.decode_jpeg(image, channels=3)
  img = scale(img)
  return np.expand_dims(img, axis=0)

path = st.text_input('Enter Image URL to Classify... (format preferred is like the example URL in .jpg or .png)', 'https://solarsystem.nasa.gov/system/stellar_items/list_view_images/6_mars_480x320.jpg')

if path is not None:
  content = requests.get(path).content

  st.write("Predicted Class :") 
  with st.spinner('classifying.....'):
    label =np.argmax(model.predict(decode_img(content) ), axis=1)
    st.write(classes [label[0]]) 
  st.write("")

  image = Image.open(BytesIO(content))
  st.image(image, use_column_width=True)


Writing UniverseClassifier.py


Ngrok token

In [ ]:
! ngrok authtoken 1s8DDstioR58vDNh3wpi8aAQv6s_7VrZRQ5mqnnWr86VkAa6o

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Using ngrok to expose the app to the internet with a public url

In [ ]:
from pyngrok import ngrok

url = ngrok.connect(port=8501)
url

<NgrokTunnel: "http://2bf3bb8c4a22.ngrok.io" -> "http://localhost:80">

Running Streamlit app

In [ ]:
!streamlit run --server.port 80 UniverseClassifier.py >/dev/null

2021-05-06 16:56:07.016 An update to the [server] config option section was detected. To have these changes be reflected, please restart streamlit.
2021-05-06 16:56:07.402081: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-06 16:56:09.420631: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-06 16:56:09.475703: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-05-06 16:56:09.559884: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-05-06 16:56:09.559958: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (7f7c25af1da3): /proc/driver/nvidia/version does not exist
2021-05-06 16:56:09.561451: I tensorflow/compiler/jit/xla_gpu_de